# Imports

In [1]:
#Enable matplotlib to display in jupyter notebook & import it
%matplotlib inline

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim #used in filling missing zipcodes


# Read and Clean Listings.csv

In [2]:
#listings.csv READING

LISTINGS = 'data/listings.csv'

#Choose which columns from the csv to read in.
listings_cols = ['id',
                'host_id',
                'neighbourhood_cleansed',
                'zipcode',
                'latitude',
                'longitude',
                'property_type',
                'room_type',
                'accommodates',
                'bathrooms',
                'amenities',
                'price',
                'cleaning_fee',
                'number_of_reviews',
                'first_review',
                'review_scores_rating',
                'review_scores_accuracy',
                'review_scores_cleanliness',
                'review_scores_checkin',
                'review_scores_communication',
                'review_scores_location',
                'review_scores_value',
                'calculated_host_listings_count',
                'reviews_per_month',
                'bedrooms',
                'beds',
                'cancellation_policy',
                'instant_bookable',
                'minimum_nights'] 

#Read in data from the csv
listings = pd.read_csv(LISTINGS, usecols=listings_cols)

#Rename any Columns as needed
rename_dict = {'id':'listing_id',
              'price':'listed_price'}

listings.rename(columns = rename_dict, inplace=True)

#use listing_id as index
listings.set_index('listing_id', inplace=True)



#############################
#         Cleaning          #
#############################

# 'zipcode' ##########
#Paste this in to zipcode section of cleaning
def latLonToZip(lat, lon):
    '''Take in a latitude and longitude and return the zipcode for that location'''
    geolocator = Nominatim()
    try:
        location = geolocator.reverse(str(lat)+','+str(lon))
        z = re.compile('(\s)([0-9]{5})(,\sUnited)')
        return z.findall(location[0])[0][1]
    except:
        print(str(lat)+','+str(lon),'-----',location)
        return np.nan
    
    print(str(lat)+','+str(lon),'-----',location)
    return np.nan
#Find all missing zippcodes : missing_zipcodes
missing_zipcodes = listings[listings.zipcode.isnull()].copy()

#update rows that are missing zipcodes using latLonToZip to fill missin
listings.zipcode.update(missing_zipcodes.apply(lambda x: latLonToZip(x['latitude'], x['longitude']), axis=1))

#Remove 'zip+4' part of any zipcode 
listings.zipcode = listings.zipcode.apply(lambda x: x[:5])

# 'price' --> 'listed_price' ##########
listings.listed_price = listings.listed_price.replace('[^0-9.]+','',regex=True).astype(float)

# 'cleaning_fee' ##########
listings.cleaning_fee = listings.cleaning_fee.replace('[^0-9.]+','',regex=True).astype(float)
listings.cleaning_fee.fillna(0, inplace = True)

# 'first_review' ##########
listings.first_review = pd.to_datetime(listings.first_review)

# 'amenities' ##########
listings.amenities = listings.amenities.replace('[^\w,\s/]+','',regex=True).apply(lambda x: x.split(','))

# 'instant_bookable' ##########
listings.instant_bookable.replace({'f':False,'t':True}, inplace=True)

# Read and Clean Calendar.csv

In [3]:
#Calendar.csv READING

CALENDAR = 'data/calendar.csv'

#Read in all columns from calendar.csv : listing_id, date, available, price
calendar = pd.read_csv(CALENDAR)


#############################
#         Cleaning          #
#############################

# 'date' ##########
calendar.date = pd.to_datetime(calendar.date)

# 'available' ##########
calendar.available.replace({'f':False,'t':True}, inplace=True)

# 'price' ##########
calendar.price = calendar.price.replace('[^0-9.]+','',regex=True).astype(float)

#############################
#         Augmenting        #
#############################

#create column to represent the day of the week for each date
calendar['day_of_week'] = calendar.date.dt.dayofweek

#Fill in missing price values for each listing using mean value for day of week from that listing
calendar.price.fillna(calendar.groupby(['listing_id','day_of_week'])['price'].transform("mean"), inplace=True)

#create column for revenue generate by property (all prices for occupied days are modeled from mean)
calendar['day_revenue'] = np.where(calendar.available, 0.0, calendar.price)

#Save to Pickle because it preserves the index and types
calendar.to_pickle('data/calendar_cleaned.pkl')

# Using Calendar to Augment Listings DataFrame

In [4]:
listings = pd.read_pickle('data/listings_cleaned.pkl')
calendar = pd.read_pickle('data/calendar_cleaned.pkl')

#Create a list of calendars seperated into 4 quarters
quarter_dates = ['2016-09-06','2016-12-06','2017-03-06','2017-06-06','2017-09-06']
q_cal = [calendar[calendar.date.isin(pd.date_range(quarter_dates[n], quarter_dates[n+1]))] for n in range(4)]

#Revnue Per Quarter
for n,q in enumerate(q_cal):
    listings = listings.join(q_cal[n].groupby('listing_id').day_revenue.sum()).rename(columns={'day_revenue':'q'+str(n+1)+'_revenue'})

#Occupancy Per Quarter
for n,q in enumerate(q_cal):
    q_len = len(pd.date_range(quarter_dates[n], quarter_dates[n+1]))
    listings = listings.join((q_len - q_cal[n].groupby('listing_id').available.sum())/q_len).rename(columns={'available':'q'+str(n+1)+'_occupancy_rate'})

listings.to_pickle('data/listing_cleaned.pkl')

# Creating Amenities Dummy Variables

In [188]:
amenities = list(set([item for item_list in listings.amenities for item in item_list]))
for val in ['translation missing enhosting_amenity_49','translation missing enhosting_amenity_50', '']:
    amenities.remove(val)

amn_frame = pd.DataFrame(index = listings.index)

#create the dummy for each amenity and rename the column as you go
for amn in amenities:
    amn_frame = amn_frame.join(listings.amenities.apply(lambda amns: amn in amns)).rename(columns={'amenities':amn})

listings['analysis_table'] = listings.index
listings['analysis_table'] = pd.DataFrame(listings.analysis_table.map(lambda x: amn_frame.loc[x]))


In [190]:
listings['amenities'].head()

listing_id
12147973    [TV, Wireless Internet, Kitchen, Free Parking ...
3075044     [TV, Internet, Wireless Internet, Air Conditio...
6976        [TV, Cable TV, Wireless Internet, Air Conditio...
1436513     [TV, Internet, Wireless Internet, Air Conditio...
7651065     [Internet, Wireless Internet, Air Conditioning...
Name: amenities, dtype: object

In [191]:
amn_frame

,Elevator in Building,Wheelchair Accessible,Hot Tub,TV,Paid Parking Off Premises,Washer / Dryer,Essentials,Dogs,Internet,Lock on Bedroom Door,...,Hangers,First Aid Kit,Wireless Internet,Indoor Fireplace,Free Parking on Premises,Family/Kid Friendly,Smoke Detector,Cable TV,Hair Dryer,Kitchen
listing_id,,,,,,,,,,,,,,,,,,,,,
12147973,False,False,False,True,False,False,True,True,False,False,...,False,False,True,False,True,True,True,False,False,True
3075044,False,False,False,True,False,False,True,True,True,True,...,True,False,True,False,False,True,True,False,True,True
6976,False,False,False,True,False,False,True,False,False,True,...,False,True,True,False,True,False,True,True,False,True
1436513,False,False,False,True,False,False,True,False,True,False,...,True,True,True,True,True,False,True,False,True,True
7651065,False,False,False,False,False,False,True,False,True,False,...,True,True,True,False,False,False,True,False,True,True
12386020,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,True,True,True,True,False,True
5706985,False,False,False,True,False,False,True,False,True,False,...,False,True,True,False,True,False,True,False,False,True
2843445,False,False,False,True,False,False,True,True,True,False,...,True,True,True,True,True,False,True,False,True,False
753446,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,True,True,False,True,False


# Creating Other Dummy Variables

In [64]:
#Taking
other_dummies = pd.DataFrame()
for i in ['instant_bookable', 'cancellation_policy', 'property_type', 'room_type']:
    other_dummies = pd.concat([other_dummies, pd.get_dummies(listings[i])], axis = 1)

#Other dummy Name Columns
c1 = [''.join(['instant_bookable_', str(other_dummies.columns[i]).strip()]) for i in range(0,2)]
c2 = [''.join(['cancel_pol_', str(other_dummies.columns[i]).strip()]) for i in range(2,6)]
c3 = [''.join(['Prop_type_', str(other_dummies.columns[i]).strip()]) for i in range(6,19)]
c4 = [''.join(['Room_type_', str(other_dummies.columns[i]).strip()]) for i in range(19,22)]

odc = c1 + c2 + c3 + c4
other_dummies.columns = odc
other_dummies.head()

,instant_bookable_False,instant_bookable_True,cancel_pol_flexible,cancel_pol_moderate,cancel_pol_strict,cancel_pol_super_strict_30,Prop_type_Apartment,Prop_type_Bed & Breakfast,Prop_type_Boat,Prop_type_Camper/RV,...,Prop_type_Entire Floor,Prop_type_Guesthouse,Prop_type_House,Prop_type_Loft,Prop_type_Other,Prop_type_Townhouse,Prop_type_Villa,Room_type_Entire home/apt,Room_type_Private room,Room_type_Shared room
listing_id,,,,,,,,,,,,,,,,,,,,,
12147973,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3075044,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6976,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1436513,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7651065,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [86]:
#Important KPI metrics have ~20% Nas
kpi = ['reviews_per_month', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location']
Review_Score_NA = pd.DataFrame([[x, listings[x].isnull().sum()/len(listings[x])] for x in kpi])
Review_Score_NA.columns =['review_score_metrics', '%_of_column_NA']
Review_Score_NA

,review_score_metrics,%_of_column_NA
0,reviews_per_month,0.210879
1,review_scores_rating,0.226778
2,review_scores_accuracy,0.229568
3,review_scores_cleanliness,0.228173
4,review_scores_checkin,0.228731
5,review_scores_communication,0.228173
6,review_scores_location,0.229289


# Subsetting By Audience

In [108]:
#Creating a specific subset of columns as attributes to analyze between audiences
audienceData = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'listed_price', \
                'cleaning_fee', 'minimum_nights', 'number_of_reviews', 'review_scores_rating', \
                'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', \
                'review_scores_communication', 'review_scores_location', 'review_scores_value', \
                'calculated_host_listings_count', 'reviews_per_month']

# Merging Dataframes

In [146]:
#Audience is a dataframe used for audience analysis, concatenated from listings & dummies
audience = pd.concat([listings[audienceData], other_dummies, amn_frame], axis=1)

#Creating Audience Segmentation variables defined based on the calculated host listings count
audience['user_type'] = audience.calculated_host_listings_count.apply(lambda x:  'Sharers' if x <= 2 else 'Businesses')
audience['user_count'] = 1

audience.select_dtypes(include=['bool']).astype(int, inplace = True)
audience.columns = [i.replace(" ", "_") for i in audience.columns]
audience.head()

,accommodates,bathrooms,bedrooms,beds,listed_price,cleaning_fee,minimum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,...,Wireless_Internet,Indoor_Fireplace,Free_Parking_on_Premises,Family/Kid_Friendly,Smoke_Detector,Cable_TV,Hair_Dryer,Kitchen,user_type,user_count
listing_id,,,,,,,,,,,,,,,,,,,,,
12147973,4,1.5,2.0,3.0,250.0,35.0,2,0,NaN,NaN,...,True,False,True,True,True,False,False,True,Sharers,1
3075044,2,1.0,1.0,1.0,65.0,10.0,2,36,94.0,10.0,...,True,False,False,True,True,False,True,True,Sharers,1
6976,2,1.0,1.0,1.0,65.0,NaN,3,41,98.0,10.0,...,True,False,True,False,True,True,False,True,Sharers,1
1436513,4,1.0,1.0,2.0,75.0,50.0,1,1,100.0,10.0,...,True,True,True,False,True,False,True,True,Sharers,1
7651065,2,1.5,1.0,2.0,79.0,15.0,2,29,99.0,10.0,...,True,False,False,False,True,False,True,True,Sharers,1


# Audience Attributes

In [183]:
#For the variables of interest that should be averaged per listing ID
audienceUser = audience.groupby('user_type', as_index = False)['user_count'].sum()
audienceMeanValues = audience.groupby('user_type', as_index = False)[list(audience.columns[0:15])].mean().round(2)
audienceMeans = pd.merge(audienceUser, audienceMeanValues, on = 'user_type').T
audienceMeans

,0,1
user_type,Businesses,Sharers
user_count,1351,2234
accommodates,3.29,2.89
bathrooms,1.33,1.16
bedrooms,1.3,1.23
beds,1.69,1.56
listed_price,185.66,166.83
cleaning_fee,86.51,54.07
minimum_nights,4.01,2.67
number_of_reviews,21.13,17.78


In [179]:
#For the variables of interest that should be summed, and then calculated as a % of user_type
audienceSumValues = audience.groupby('user_type', as_index = False)[list(audience.columns[17:84])].sum()
audienceSums = pd.merge(audienceUser, audienceSumValues, on = 'user_type')
audienceSums.ix[2, :] = audienceSums.sum()
audienceSums.ix[2, 0] = 'Total'
audienceSums.ix[0, 2:] = audienceSums.ix[0, 2:]/audienceSums.ix[0, 1]
audienceSums.ix[1, 2:] = audienceSums.ix[1, 2:]/audienceSums.ix[1, 1]
audienceSums.round(2).T

,0,1,2
user_type,Businesses,Sharers,Total
user_count,1351,2234,3585
instant_bookable_False,0.78,0.86,2991
instant_bookable_True,0.22,0.14,594
cancel_pol_flexible,0.11,0.38,999
cancel_pol_moderate,0.2,0.29,919
cancel_pol_strict,0.63,0.33,1582
cancel_pol_super_strict_30,0.06,0,85
Prop_type_Apartment,0.74,0.72,2612
Prop_type_Bed_&_Breakfast,0.02,0.01,41
